In [5]:
%run ../nli/config.py
%run ../nli/nli_finetune.py
%run ../nli/mnli.py
%run ../nli/nli_dataset.py

In [18]:
import pandas as pd

import torch
import transformers
import torchmetrics
import pytorch_lightning as pl

from transforms import ClinicalSynonymSubstitution

In [ ]:
# transform=ClinicalSynonymSubstitution()
# text="Patient has elevated BUN"
# transformed_text=transform(text)
# transformed_text

In [ ]:
# CONFIG["MODEL_SAVE_NAME"]="" # Make changes here to specify different model.
# CONFIG["CHECKPOINT_DIR"]="" # Make changes here to specify corresponding model checkpoint.

In [ ]:
_=pl.seed_everything(CONFIG['SEED'])

In [ ]:
trained_model = NLIFineTuningModel.load_from_checkpoint(checkpoint_path=args.model_checkpoint,  # model checkpoint path
                                                        num_labels=CONFIG['NUM_CLASSES'],
                                                        model_name_or_path=CONFIG['MODEL_NAME_OR_PATH'])
trained_model.freeze()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    CONFIG['MODEL_NAME_OR_PATH'])

In [ ]:
test_df = mnli_df('test')

In [ ]:
test_dataset = NLIDataset(max_len=CONFIG['MAX_LEN'],
                          tokenizer=tokenizer,
                          sentence1=test_df[CONFIG['sentence1']].values,
                          sentence2=test_df[CONFIG['sentence2']].values,
                          labels=test_df[CONFIG['labels']].values,
                          transforms=ClinicalSynonymSubstitution()
                         )

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                                      batch_size=CONFIG['VAL_BS'],
                                                      shuffle=False,
                                                      num_workers=CONFIG['NUM_WORKERS'])

In [ ]:
test_metrics=torchmetrics.MetricCollection([
            torchmetrics.Accuracy(),
            torchmetrics.F1(num_classes=CONFIG['NUM_CLASSES'], average='macro')])

In [ ]:
for batch in test_dataloader:
    loss, logits = self(batch)[:2]
    predictions = torch.argmax(logits, dim=1)
    test_metrics(predictions, batch['labels'])
test_metrics.compute()